In [1]:
from citipy import citipy
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from scipy.stats import linregress
import time


import sys
sys.path.append('..')
from config import weather_api_key

# Retrieve Weather Data

### Generate 2000 random coordinates, retrieve their nearest cities

In [2]:
# create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

# add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [3]:
# ensure we have only have unique cities, i.e. no duplicates
cities = {citipy.nearest_city(c[0], c[1]).city_name for c in coordinates}
print(f'Total number of cities: {len(cities)}')

Total number of cities: 773


### Retrieve weather information for the `cities` generated above

In [4]:
# create the base url
base_url = 'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=' + weather_api_key

In [5]:
# create an empty list to hold the weather data
city_data = []

# loop through all cities in our list
for i, city in enumerate(cities):
    # prevent hitting API ratelimits by sleeping every 50 cities
    if i%50 == 0 and i >= 50:
        time.sleep(60)
    
    # create endpoint url with each city
    city_url = base_url + '&q=' + city.replace(' ', '+')

    try: 
        # parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        
        # parse out the needed data
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_description = city_weather['weather'][0]['description']
        
        #convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        
        # append the information to the city_data list
        city_data.append(
            {
                'City': city.title(),
                'Lat': city_lat,
                'Lng': city_lng,
                'Max Temp': city_max_temp,
                'Humidity': city_humidity,
                'Cloudiness': city_clouds,
                'Wind Speed': city_wind,
                'Country': city_country,
                'Date': city_date,
                'Description': city_description
            }
        )
    except:
        print(f'City not found: {city}. Skipping...')
        pass

# indicate that data loading is complete
print('----------------------------')
print('Data retrieval complete     ')
print('----------------------------')

City not found: karaul. Skipping...
City not found: richard toll. Skipping...
City not found: saryshagan. Skipping...
City not found: bolungarvik. Skipping...
City not found: yomitan. Skipping...
City not found: mrirt. Skipping...
City not found: tapaua. Skipping...
City not found: kamenskoye. Skipping...
City not found: panjwin. Skipping...
City not found: tsihombe. Skipping...
City not found: azimur. Skipping...
City not found: masjed-e soleyman. Skipping...
City not found: viligili. Skipping...
City not found: maymanak. Skipping...
City not found: barentsburg. Skipping...
City not found: safwah. Skipping...
City not found: stoyba. Skipping...
City not found: bargal. Skipping...
City not found: samalaeulu. Skipping...
City not found: tumannyy. Skipping...
City not found: barawe. Skipping...
City not found: lolua. Skipping...
City not found: chagda. Skipping...
City not found: temaraia. Skipping...
City not found: warqla. Skipping...
City not found: samusu. Skipping...
City not found:

### Save the city data as a Pandas DataFrame & export as a csv

In [6]:
# convert the list of dictionaries to a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.to_csv('WeatherPy_Database.csv', index_label='City_ID')

In [7]:
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,Cherskiy,68.7500,161.3000,20.68,94,100,9.46,RU,2022-05-08 19:39:41,light snow
1,Camana,-16.6228,-72.7111,70.47,73,52,10.63,PE,2022-05-08 19:39:42,broken clouds
2,Sakakah,29.9697,40.2064,71.17,16,51,7.02,SA,2022-05-08 19:39:42,broken clouds
3,Seia,40.4151,-7.7086,62.10,62,6,1.90,PT,2022-05-08 19:39:43,clear sky
4,Belmonte,-15.8631,-38.8828,75.92,85,90,10.69,BR,2022-05-08 19:39:43,overcast clouds
5,Kurayoshi,35.4333,133.8167,54.23,70,100,7.36,JP,2022-05-08 19:39:44,overcast clouds
6,Vanavara,60.3400,102.2797,25.97,88,100,10.13,RU,2022-05-08 19:39:45,overcast clouds
7,Waitara,-38.9250,174.2500,64.49,81,100,15.26,NZ,2022-05-08 19:39:45,overcast clouds
8,Rincon,32.2960,-81.2354,72.86,56,75,5.75,US,2022-05-08 19:39:46,broken clouds
9,Korem,12.5058,39.5228,60.39,58,1,2.10,ET,2022-05-08 19:39:46,clear sky
